## 准备数据

In [34]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [36]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [39]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1=tf.compat.v1.get_variable(shape=(784,1000),name="W1",initializer=tf.compat.v1.random_normal_initializer)
        self.b1=tf.compat.v1.get_variable(shape=(1,1000),name="b1",initializer=tf.compat.v1.random_normal_initializer)
        self.W2=tf.compat.v1.get_variable(shape=(1000,666),name="W2",initializer=tf.compat.v1.random_normal_initializer)
        self.b2=tf.compat.v1.get_variable(shape=(1,666),name="b2",initializer=tf.compat.v1.random_normal_initializer)
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x=tf.reshape(x,(-1,784))
        hidden = tf.nn.relu(tf.matmul(x,self.W1)+self.b1)
        logits = tf.matmul(hidden,self.W2)+self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [42]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [47]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 169.8196 ; accuracy 0.15536667
epoch 1 : loss 165.22043 ; accuracy 0.16343333
epoch 2 : loss 160.81456 ; accuracy 0.17125
epoch 3 : loss 156.5789 ; accuracy 0.1782
epoch 4 : loss 152.50114 ; accuracy 0.18465
epoch 5 : loss 148.58105 ; accuracy 0.1921
epoch 6 : loss 144.80977 ; accuracy 0.19923334
epoch 7 : loss 141.17763 ; accuracy 0.206
epoch 8 : loss 137.68057 ; accuracy 0.21256667
epoch 9 : loss 134.31212 ; accuracy 0.2189
epoch 10 : loss 131.07133 ; accuracy 0.22416666
epoch 11 : loss 127.960945 ; accuracy 0.23016667
epoch 12 : loss 124.98151 ; accuracy 0.23576666
epoch 13 : loss 122.12473 ; accuracy 0.24095
epoch 14 : loss 119.3839 ; accuracy 0.24606666
epoch 15 : loss 116.76195 ; accuracy 0.25138333
epoch 16 : loss 114.2518 ; accuracy 0.25643334
epoch 17 : loss 111.847176 ; accuracy 0.26108333
epoch 18 : loss 109.537155 ; accuracy 0.26606667
epoch 19 : loss 107.31125 ; accuracy 0.27026665
epoch 20 : loss 105.16261 ; accuracy 0.27453333
epoch 21 : loss 103.08731 ; a